# Mexican federal budget pre-processing pipeline

## Instructions

To you run the notebook:

1. choose a unique `ITERATION_LABEL` for each pipeline run
2. specify and describe your input files (`INPUT_FILES`)
3. make sure your column mapping (`COLUMN_ALIASES`) is correct
3. run the whole notebook by clicking on __Kernel > Restart & Run All__

## Settings

Choose a unique iteration label for each pipeline run.

In [1]:
ITERATION_LABEL = 'iteration-9-development'

Put your input files inside the `pipeline.in` folder and describe them here.

In [2]:
INPUT_FILES = {
    2010: {'name': 'Cuenta_Publica_2010.csv', 'encoding': 'windows-1252'},
    2011: {'name': 'Cuenta_Publica_2011.csv', 'encoding': 'windows-1252'},
    2012: {'name': 'Cuenta_Publica_2012.csv', 'encoding': 'windows-1252'},
    2013: {'name': 'Cuenta_Publica_2013.csv', 'encoding': 'windows-1252'},
    2014: {'name': 'Cuenta_Publica_2014.csv', 'encoding': 'windows-1252'},
    2015: {'name': 'Cuenta_Publica_2015.csv', 'encoding': 'windows-1252'},
    2016: {'name': '2016_2T_Gasto_OS.csv', 'encoding': 'windows-1252'} # cp850 for the original 2016 file
}

If your input files don't all have the same column names, define your mapping here. 

In [3]:
COLUMN_ALIASES = {
    'Actividad Institucional': ['AI'],
    'Adefas': ['ADEFAS'],
    'Aprobado': [
        'PEF_2016',
        'Importe Presupuesto de Egresos de la Federación',
        'Importe Presupuesto de Egresos de la Federación (PEF)'
    ],
    'Ciclo': None,
    'Clave de cartera': ['CLAVE_CARTERA'],
    'Descripción de Fuente de Financiamiento': ['FUENTE_FINAN_DESCRIPCION'],
    'Descripción de Función': ['FUNCIONL_DESCRIPCION'],
    'Descripción de Grupo Funcional': [
        'Descripción de Finalidad',
        'GRUPO_FUN_DESCRIPCION',
        'Descripción de Grupo Funcional'
    ],
    'Descripción de Objeto del Gasto': ['CONCEPTO_DESCRIPCION'],
    'Descripción de Programa Presupuestario': ['PROGR_PRES_DESCRIPCION'],
    'Descripción de Ramo': ['RAMO_DESCRIPCION'],
    'Descripción de Reasignacion': ['REASIGNACION_DESCRIPCION'],
    'Descripción de Subfunción': ['SUBFUNCIONL_DESCRIPCION', 'Descripción de subfunción'],
    'Descripción de Tipo de Gasto': ['TIPO_GASTO_DESCRIPCION'],
    'Descripción de Unidad Responsable': ['UNIDAD_DESCRIPCION'],
    'Descripción de la Actividad Institucional': [
        'ACTIVIDAD_INST_DESCRIPCION',
        'Descripción de Actividad Institucional'
    ],
    'Descripción de Entidad Federativa': ['Descripción de la entidad federativa', 'ENTIDAD_FED_DESCRIPCION'],
    'Descripción de la modalidad del programa presupuestario': [
        'MODALIDAD_DESCRIPCION',
        'Descripción del Identificador del Programa Presupuestario',
        'Descripción del Identificador de Programa Presupuestario'
    ],
    'Devengado': None,
    'Ejercicio': None,
    'Ejercido': None,
    'Entidad Federativa': ['EF'],
    'Fuente de Financiamiento': ['FF', 'Fuente de Finaciamiento'],
    'Función': ['FN'],
    'Grupo Funcional': [
        'Finalidad', 'GF', 'Grupo Funcional'
    ],
    'Modalidad del Programa presupuestario': [
        'MOD',
        'Identificador de Programa Presupuestario',
        'Identificador del Programa Presupuestario'
    ],
    'Modificado': None,
    'Objeto del Gasto': ['CONCEPTO'],
    'Pagado': None,
    'Programa Presupuestario': ['PP'],
    'Ramo': ['RAMO'],
    'Reasignacion': ['RA'],
    'Subfunción': ['SF'],
    'Tipo de Gasto': ['TG'],
    'Unidad Responsable': ['UNIDAD'],
    'Capitulo': None,
    'Concepto': None,
    'Partida Genérica': None,
    'Partida Específica': None,
    'Descripción de Capitulo': None,
    'Descripción de Concepto': None,
    'Descripción de Partida Genérica': None,
    'Descripción de Partida Específica': ['Descripcion de Partida Específica'],    
}

The following hierarchical categories will have IDs prefixed with the parent categories:

In [4]:
HIERARCHIES = {
    'functional': [
        'Grupo Funcional', 
        'Función', 
        'Subfunción', 
        'Actividad Institucional'
    ],
    'administrative': [
        'Ramo', 
        'Unidad Responsable'
    ],
    'activities': [
        'Modalidad del Programa presupuestario', 
        'Programa Presupuestario'
    ],
}

The following columns are unsused and removed at the end of the pipeline:

In [5]:
REMOVE_OUTPUT_COLUMNS = [
    'Reasignacion',
    'Objeto del Gasto',
    'Descripción de Reasignacion',
    'Descripción de Objeto del Gasto'
]

In [6]:
REMOVE_INPUT_COLUMNS = {
    2016: [
        'Adefas',
        'Partida Específica',
        'Partida Genérica',
        'Descripción de Partida Genérica',
        'Descripcion de Partida Específica',
        'Ejercicio',
        'Devengado',
        'Ejercido',
    ]
}

That's it. Now just run the notebook from beginning to end.

## Imports

In [7]:
from sys import stdout
from pandas import read_csv, concat, DataFrame, ExcelWriter, ExcelFile, Series
from numpy import nan, isnan
from os.path import join, isdir
from os import mkdir
from json import dumps, loads
from pprint import pprint

## Configuration

In [8]:
BASENAME = 'mexican_federal_budget'
INPUT_FOLDER = 'pipeline.in'
OUTPUT_FOLDER = 'pipeline.out'
ITERATION_FOLDER = join(OUTPUT_FOLDER, ITERATION_LABEL)
MERGED_FILE = join(ITERATION_FOLDER, BASENAME + '.merged.csv')
CATALOGS_FOLDER = 'objeto_del_gasto.catalog'
CATALOGS_FILE = 'objeto_del_gasto.catalog.xlsx'

In [9]:
if isdir(ITERATION_FOLDER):
    raise ValueError('Please enter a unique iteration label')
    
mkdir(ITERATION_FOLDER)

## Encoding inspection

Detect the file encodings of the input files using the `cChardet` utility library. __Warning:__ it's not always accurate. This is meant only as an indication only. In the end, encodings will be taken from `INPUT_FILES`.

In [10]:
def detect_encodings():
    """Detect CSV file encoding with the cChardet library"""

    try:
        import cchardet as chardet
    except ImportError:
        cChardet = 'https://github.com/PyYoshi/cChardet'
        print('Encoding inspection skipped: install %s', cChardet)
        return

    results = {}
    results_file = join(OUTPUT_FOLDER, ITERATION_LABEL, 'encodings.detected.json')
    
    for year, file in sorted(INPUT_FILES.items()):
        datafile = join(INPUT_FOLDER, file['name'])
        
        with open(datafile, 'rb') as f:
            text = f.read()
            
        result = chardet.detect(text)
        results.update({year: result})
        print(year, 'Inspected', file['name'], result)
    
    with open(results_file, 'w+') as json:
        json.write(dumps(results, indent=4))
        print('\nSaved encoding detection report to', results_file)
        
# detect_encodings()

## Load files

In [11]:
def read_columns(file, encoding):
    """Return clean CSV file headers"""
    
    with open(file, encoding=encoding) as csv:
        header = csv.readline()
        return header.replace('\n', '').split(',')

In [12]:
def force_strings(columns):
    """Return string enforcement for each column of a CSV file"""
    
    for column in columns:
        yield column, str

In [13]:
def load_csv_files():
    """Load raw data (CSV) files"""
    
    batch = {}
    
    for year, file in sorted(INPUT_FILES.items()):
        filepath = join(INPUT_FOLDER, file['name'])
        column_names = read_columns(filepath, file['encoding'])
        column_types = dict(force_strings(column_names))
        
        batch[year] = read_csv(filepath, encoding=file['encoding'], dtype=column_types)
        print('Loaded', file['name'], 'with encoding', file['encoding'])
    
    print()
    stdout.flush()

    for year in sorted(INPUT_FILES.keys()):
        if year in REMOVE_INPUT_COLUMNS:
            for column in REMOVE_INPUT_COLUMNS[year]:
                try:
                    del batch[year][column]
                    print(year, 'deleted', column)
                except KeyError:
                    print(year, column, 'not found in', file['name'])

        stdout.flush()

    return batch

## Clean the data

In [14]:
def strip_cell_padding(batch):
    for year in sorted(batch.keys()):
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
            batch[year][column] = batch[year][column].apply(lambda x: x.strip() if x is not nan else x)
        print(year, 'stripped cell paddings')
        stdout.flush()

In [15]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    del batch[year][column]
                    print(year, column, 'deleted')
                    stdout.flush()
                except KeyError:
                    pass  

In [16]:
def count_missing_values(batch):
    collector = {}
    table = []

    for column in get_union_of_columns(batch):
        row = {'Column': column}
        collector.update({column: []})
        
        for year in batch.keys():
            if column in batch[year].columns:
                is_empty = batch[year][column].isnull()
                empty_lines = batch[year].where(is_empty).dropna(how='all')
                collector[column].extend(empty_lines.to_dict(orient='records'))
                nb_empty_cells = len(empty_lines)
            else:
                nb_empty_cells = nan
                
            row.update({year: nb_empty_cells})
            if nb_empty_cells not in (nan, 0):
                print(year, 'found', nb_empty_cells, 'missing values in', column)

        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    empty_values_overview_table = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    
    return empty_values_overview_table, collector

In [17]:
def count_duplicates(batch):
    for year, df in sorted(batch.items()):
        nb_duplicate_lines = df.duplicated().apply(lambda x: 1 if x is True else 0).sum()
        print(year, 'found', nb_duplicate_lines, 'duplicate lines')

## Alias column names

In [18]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [19]:
from yaml import load

def load_aliases(file):
    with open(file) as yaml:
        aliases = load(yaml.read())
        return aliases

In [20]:
def map_columns_to_aliases(batch, list_of_aliases):
    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, column, 'replaced with', reference)
                            stdout.flush()
                            break  
                else:
                    print(year, 'NO ALIAS REGISTERED FOR', column)
                    stdout.flush()

In [21]:
def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    overview = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    print('Column mapping overview: done')
    return overview

## Check expenditure sums

There's a little cleaning to do on the amount columns (zeros represented by a dash). Assume thousands are seperated by a comma.

In [22]:
EXPENDITURE_COLUMNS = [
    'Ejercido', 
    'Devengado', 
    'Aprobado', 
    'Pagado', 
    'Modificado', 
    'Adefas', 
    'Ejercicio'
]
count = 0

def clean_expenditure_columns(batch):
    check_sums = []

    for column in EXPENDITURE_COLUMNS:
        row = {'Column': column}
        
        for year in sorted(batch.keys()):
            try:
                series = batch[year][column]
                
                # I'm assuming a single '-' represents zero
                series = series.apply(lambda x: '0' if x == '-' else x)
                try:
                    series = series.apply(lambda x: x.replace(',', '') if x is not nan else x)    
                except AttributeError:
                    if count < 10:
                        print(year, column)
                batch[year][column] = series.astype(float)
                check_sum = batch[year][column].sum()
                
                print(year, 'cleaned and summed', column, '=', check_sum, 'pesos')
                
            except KeyError:
                check_sum = nan
                
            row.update({year: check_sum})
        
        check_sums.append(row)

    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    return DataFrame(check_sums).reindex_axis(ordered_columns, axis=1)    

## Objeto del Gasto Column split

In [23]:
from os.path import join

def generate_catalog(file):
    
    catalog_ = {}
    catalog_file = ExcelFile(file)
    INDEX_COLUMN = 0
    
    for sheet in catalog_file.sheet_names:
        if sheet != 'Concatenated':
            name = sheet.lower().replace(' ', '_')
            output = join('objeto_del_gasto.catalog', name + '.csv')

            df = catalog_file.parse(sheet).dropna()
            index = df.columns[INDEX_COLUMN]

            df[index] =  df[index].astype(str)
            df.set_index(index, inplace=True)
            df = df.groupby(df.index).first()
            df.sort_index(inplace=True)
            
            message = 'Loaded catalog {sheet} into "{name}" ({nb} lines)'
            parameters = dict(sheet=sheet, name=name, nb=len(df))

            print(message.format(**parameters))
            catalog_[name] = df['DESCRIPCION']
    
    print()
    return catalog_

__Note!__ Years are hard coded in the script below.

In [26]:
def split_objeto_del_gasto(batch):
    catalog = generate_catalog(CATALOGS_FILE)
    missing_in_catalog = []
    
    def has_digits(n, N):
        return not isinstance(n, float) and len(n) >= N 
            

    def lookup(n, table, year):
        try:
            return catalog[table].loc[n]
        except KeyError:
            missing_in_catalog.append({'year': year, 'table': table, 'ID': n})
            return nan
        except TypeError:
            # n is nan
            return nan
    
    for year in sorted(batch.keys()):
        if year == 2016:
            print('Skipping', year, 'because the raw CSV already has the required columns')
        
        else:
            objeto = batch[year]['Objeto del Gasto'].astype(str)

            batch[year]['Capitulo'] = objeto.apply(lambda x: x[0] + '000' if x not in (nan, 'nan') else nan)
            batch[year]['Concepto'] = objeto.apply(lambda x: x[:2] + '00' if x not in (nan, 'nan') else nan)
            batch[year]['Descripción de Capitulo'] = batch[year]['Capitulo'].map(lambda x: lookup(x, 'capitulo', year))  
            batch[year]['Descripción de Concepto'] = batch[year]['Concepto'].map(lambda x: lookup(x, 'concepto', year))  
            
            nb_generica_digits = 4 if year in (2008, 2009, 2010) else 3
            
            # Skip the LAST year of the dataset (currently 2016) it has split columns already
            batch[year]['Partida Genérica'] = objeto.apply(lambda x: x[:nb_generica_digits] if has_digits(x, 4) else nan)
            batch[year]['Descripción de Partida Genérica'] = batch[year]['Partida Genérica'].map(lambda x: lookup(x, 'partida_generica', year))  
            
            if year not in (2008, 2009, 2010):
                batch[year]['Partida Específica'] = objeto.apply(lambda x: x if has_digits(x, 5) else nan)
                batch[year]['Descripción de Partida Específica'] = batch[year]['Partida Específica'].map(lambda x: lookup(x, 'partida_específica', year) if has_digits(x, 5) else nan)  
            else:
                batch[year]['Partida Específica'] = nan
                batch[year]['Descripción de Partida Específica'] = nan

            print(year, 'broke down "Objeto del Gasto" column')
        
    return DataFrame(missing_in_catalog).drop_duplicates(['ID', 'table'])

## Prefix IDs 
Disambiguating sub-categories may require prefixing their IDs with their parents' IDs.

In [27]:
def prefix_ids(batch):
    for year in batch.keys():       
        for hierarchy, levels in HIERARCHIES.items():
            prefix = batch[year]['Ciclo'].apply(lambda x: '')
            for n, level in enumerate(levels):
                dash = '.' if n > 0 else ''
                prefix = prefix + dash + batch[year][level]  
                batch[year][level] = prefix
                
                print(year, 'prefixed', hierarchy, 'level', n, level)
                stdout.flush()

## Remove unused columns

In [28]:
def remove_unused_columns(batch):
    for year, budget in batch.items():
        for column in REMOVE_OUTPUT_COLUMNS:
            try:
                del budget[column]
                print(year, 'deleted', column)
            except KeyError:
                pass

##  Pipeline

In [29]:
def do_pipeline():

    def echo_section(section):
        print('\n', section, '\n')

    echo_section('Loading files')
    datasets = load_csv_files()
    
    echo_section('Delete empty columns')
    delete_empty_columns(datasets)

    echo_section('Stripping padding from cells')
    strip_cell_padding(datasets)
    
    echo_section('Counting duplicate lines (NOT de-duplicating)')
    count_duplicates(datasets)
    
    echo_section('Mapping column to aliases')
    map_columns_to_aliases(datasets, COLUMN_ALIASES)

    echo_section('Counting missing values')
    missing_values_report, bad_records = count_missing_values(datasets)
    
    echo_section('Building column mapping overview')
    column_mapping_report = build_overview(datasets)
    
    echo_section('Cleaning expenditure columns')
    sums_report = clean_expenditure_columns(datasets)
    
    echo_section('Breaking down Objeto del Gasto column')
    missing_catalog_ids = split_objeto_del_gasto(datasets)
        
    echo_section('Prefixing IDs of certain category hierarchies')
    prefix_ids(datasets)

    echo_section('Removing unused columns')
    remove_unused_columns(datasets)

    echo_section('Saving pipeline configuration')

    reports_file = join(ITERATION_FOLDER, BASENAME + '.reports.xlsx')
    writer = ExcelWriter(reports_file)    
    missing_values_report.to_excel(writer, 'missing values', encoding='utf-8', index=False)
    column_mapping_report.to_excel(writer, 'column mapping', encoding='utf-8', index=False)
    sums_report.to_excel(writer, 'check sums', encoding='utf-8', index=False)
    missing_catalog_ids.to_excel(writer, 'missing_catalog_IDs', encoding='utf-8', index=False)    
    print('Saved 4 reports to', reports_file)    

    aliases_file = join(ITERATION_FOLDER, BASENAME + '.aliases.json')
    inputs_file = join(ITERATION_FOLDER, BASENAME + '.inputs.json')
    levels_file = join(ITERATION_FOLDER, BASENAME + '.levels.json')
    bad_records_file = join(ITERATION_FOLDER, BASENAME + '.missing.json')

    with open(bad_records_file, 'w+') as json:
        json.write(dumps(bad_records, indent=4))
        
    with open(aliases_file, 'w+') as json:
        json.write(dumps(COLUMN_ALIASES, indent=4))
        
    with open(levels_file, 'w+') as json:
        json.write(dumps(HIERARCHIES, indent=4))
        
    with open(inputs_file, 'w+') as json:
        json.write(dumps(INPUT_FILES, indent=4))
    
    print('Saved input configuration to', inputs_file)    
    print('Saved column aliases to', aliases_file) 
    print('Saved bad records (those with empty cells) to', bad_records_file)    
    print('Saved hierarchy levels used for prefixing to', levels_file) 
    
    echo_section('Pipeline run "%s" done' % ITERATION_LABEL)

    return datasets, missing_catalog_ids, column_mapping_report, missing_values_report, sums_report

## Run the pipeline

In [30]:
budgets, missing_ids, column_mapping, missing_values, sums = do_pipeline()


 Loading files 

Loaded Cuenta_Publica_2010.csv with encoding windows-1252
Loaded Cuenta_Publica_2011.csv with encoding windows-1252
Loaded Cuenta_Publica_2012.csv with encoding windows-1252
Loaded Cuenta_Publica_2013.csv with encoding windows-1252
Loaded Cuenta_Publica_2014.csv with encoding windows-1252
Loaded Cuenta_Publica_2015.csv with encoding windows-1252
Loaded 2016_2T_Gasto_OS.csv with encoding windows-1252

2016 deleted Adefas
2016 deleted Partida Específica
2016 deleted Partida Genérica
2016 deleted Descripción de Partida Genérica
2016 deleted Descripcion de Partida Específica
2016 deleted Ejercicio
2016 deleted Devengado
2016 deleted Ejercido

 Delete empty columns 

2011 Unnamed: 25 deleted
2011 Unnamed: 26 deleted
2011 Unnamed: 27 deleted
2011 Unnamed: 28 deleted
2011 Unnamed: 29 deleted
2011 Unnamed: 30 deleted
2011 Unnamed: 31 deleted
2011 Unnamed: 32 deleted
2011 Unnamed: 33 deleted
2011 Unnamed: 34 deleted
2011 Unnamed: 35 deleted
2011 Unnamed: 36 deleted
2011 Unname

In [31]:
from gc import collect
collect()

4460

In [32]:
for year, budget in budgets.items():
    filepath = MERGED_FILE.replace('merged', str(year))
    budget.to_csv(filepath, encoding='utf-8', index=False)
    print('Saved', filepath)
    stdout.flush()

Saved pipeline.out/iteration-9-development/mexican_federal_budget.2016.csv
Saved pipeline.out/iteration-9-development/mexican_federal_budget.2010.csv
Saved pipeline.out/iteration-9-development/mexican_federal_budget.2011.csv
Saved pipeline.out/iteration-9-development/mexican_federal_budget.2012.csv
Saved pipeline.out/iteration-9-development/mexican_federal_budget.2013.csv
Saved pipeline.out/iteration-9-development/mexican_federal_budget.2014.csv
Saved pipeline.out/iteration-9-development/mexican_federal_budget.2015.csv


In [33]:
merged = concat(list(budgets.values()))
merged.to_csv(MERGED_FILE, encoding='utf-8', index=False)
print('Saved merged dataset to', MERGED_FILE)    

Saved merged dataset to pipeline.out/iteration-9-development/mexican_federal_budget.merged.csv


## Quality control

In [34]:
sorted(list(budget.columns))

['Actividad Institucional',
 'Adefas',
 'Aprobado',
 'Capitulo',
 'Ciclo',
 'Clave de cartera',
 'Concepto',
 'Descripción de Capitulo',
 'Descripción de Concepto',
 'Descripción de Entidad Federativa',
 'Descripción de Fuente de Financiamiento',
 'Descripción de Función',
 'Descripción de Grupo Funcional',
 'Descripción de Partida Específica',
 'Descripción de Partida Genérica',
 'Descripción de Programa Presupuestario',
 'Descripción de Ramo',
 'Descripción de Subfunción',
 'Descripción de Tipo de Gasto',
 'Descripción de Unidad Responsable',
 'Descripción de la Actividad Institucional',
 'Descripción de la modalidad del programa presupuestario',
 'Devengado',
 'Ejercicio',
 'Entidad Federativa',
 'Fuente de Financiamiento',
 'Función',
 'Grupo Funcional',
 'Modalidad del Programa presupuestario',
 'Modificado',
 'Pagado',
 'Partida Específica',
 'Partida Genérica',
 'Programa Presupuestario',
 'Ramo',
 'Subfunción',
 'Tipo de Gasto',
 'Unidad Responsable']

In [35]:
merged.sample(n=10)

,Actividad Institucional,Adefas,Aprobado,Capitulo,Ciclo,Clave de cartera,Concepto,Descripción de Capitulo,Descripción de Concepto,Descripción de Entidad Federativa,...,Modalidad del Programa presupuestario,Modificado,Pagado,Partida Específica,Partida Genérica,Programa Presupuestario,Ramo,Subfunción,Tipo de Gasto,Unidad Responsable
71156,3.5.1.10,NaN,5000.0,2000,2013,33834,2100,Materiales y suministros,"Materiales de administracion, emision de docum...",Sinaloa,...,K,NaN,NaN,21601,216,K.31,9,3.5.1,3,9.645
144052,2.6.7.16,NaN,31360.0,3000,2016,0,3100,Servicios generales,Servicios basicos,Hidalgo,...,P,12207.0,12207.0,NaN,NaN,P.13,47,2.6.7,1,47.AYB
466,1.8.3.1,NaN,0.0,3000,2010,NaN,3500,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",NaN,...,O,NaN,NaN,NaN,3505,O.1,2,1.8.3,1,2.113
109761,3.8.1.14,0.0,1986119.0,1000,2014,0,1300,Servicios personales,Remuneraciones adicionales y especiales,Distrito Federal,...,E,1986119.0,1986119.0,13407,134,E.21,11,3.8.1,1,11.L4J
119532,2.6.8.12,0.0,534540.0,3000,2014,0,3200,Servicios generales,Servicios de arrendamiento,Distrito Federal,...,P,0.0,0.0,32503,325,P.13,12,2.6.8,1,12.E00
62240,3.5.1.10,NaN,11600.0,3000,2013,38045,3500,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Nuevo León,...,K,NaN,NaN,35501,355,K.31,9,3.5.1,3,9.639
98520,3.1.1.7,0.0,425218.0,3000,2014,0,3200,Servicios generales,Servicios de arrendamiento,Distrito Federal,...,E,354349.0,354349.0,32503,325,E.5,10,3.1.1,1,10.LAT
125196,2.5.3.5,NaN,30000000.0,4000,2012,0,4300,"Transferencias, asignaciones, subsidios y otra...",Subsidios y subvenciones,Tlaxcala,...,U,NaN,NaN,43801,438,U.6,11,2.5.3,1,11.511
54289,3.5.1.3,NaN,0.0,2000,2012,7096270022,2900,Materiales y suministros,"Herramientas, refacciones y accesorios menores",Chiapas,...,K,NaN,NaN,29401,294,K.3,9,3.5.1,3,9.627
139930,3.1.2.5,0.0,60060.0,1000,2014,0,1500,Servicios personales,Otras prestaciones sociales y económicas,Tlaxcala,...,E,45329.0,45329.0,15401,154,E.3,14,3.1.2,1,14.149


In [36]:
objeto_breakdown = [
    'Ciclo', 
    'Capitulo', 'Concepto', 
    'Partida Específica', 
    'Partida Genérica'
]
merged[objeto_breakdown].sample(n=20)

,Ciclo,Capitulo,Concepto,Partida Específica,Partida Genérica
22933,2016,1000,1300,NaN,NaN
51036,2015,3000,3100,31401,314
164406,2012,6000,6200,62302,623
93298,2014,3000,3700,37204,372
62114,2012,2000,2100,21101,211
26653,2015,5000,5600,56201,562
81255,2014,3000,3200,32201,322
194144,2013,2000,2200,22106,221
223215,2013,2000,2400,24501,245
245423,2016,1000,1400,NaN,NaN


In [37]:
print('Total: missing', len(missing_ids), 'catalog IDs to breakdown the "Objeto del Gasto" column')
print('Tables:', dict(missing_ids.groupby('table').count()['ID']))
print('Years:', dict(missing_ids.groupby('year').count()['ID']))
missing_ids.sample(n=20)

Total: missing 543 catalog IDs to breakdown the "Objeto del Gasto" column
Tables: {'partida_generica': 512, 'concepto': 7, 'partida_específica': 24}
Years: {2010: 519, 2012: 22, 2013: 1, 2015: 1}


,ID,table,year
4697,4211,partida_generica,2010
131645,15909,partida_específica,2012
2708,3827,partida_generica,2010
105654,4302,partida_generica,2010
119497,8222,partida_generica,2010
131649,34102,partida_específica,2012
131643,15907,partida_específica,2012
9632,3303,partida_generica,2010
9141,3806,partida_generica,2010
2782,3105,partida_generica,2010


In [38]:
missing_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543 entries, 0 to 131661
Data columns (total 3 columns):
ID       543 non-null object
table    543 non-null object
year     543 non-null int64
dtypes: int64(1), object(2)
memory usage: 17.0+ KB


In [39]:
writer = ExcelWriter('test4.xlsx')    
no_dups = missing_ids.drop_duplicates('ID')
print(len(no_dups))
no_dups.to_excel(writer, 'sheet_name', encoding='utf-8', index=False)    

543


In [40]:
DataFrame([{'a': 1, "b": 2, 'c': 3}, {'a': 2, "b": 2, 'c': 4}]).drop_duplicates('b')

,a,b,c
0,1,2,3


In [41]:
column_mapping

,Column,2010,2011,2012,2013,2014,2015,2016
0,Descripción de la Actividad Institucional,True,True,True,True,True,True,True
1,Descripción de Subfunción,True,True,True,True,True,True,True
2,Descripción de Entidad Federativa,False,False,True,True,True,True,True
3,Entidad Federativa,False,False,True,True,True,True,True
4,Objeto del Gasto,True,True,True,True,True,True,False
5,Grupo Funcional,True,True,True,True,True,True,True
6,Fuente de Financiamiento,True,True,True,True,True,True,True
7,Descripción de Concepto,False,False,False,False,False,False,True
8,Pagado,False,False,False,False,True,True,True
9,Descripción de Tipo de Gasto,True,True,True,True,True,True,True


In [42]:
missing_values

,Column,2010,2011,2012,2013,2014,2015,2016
0,Descripción de la Actividad Institucional,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Descripción de Subfunción,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Descripción de Entidad Federativa,NaN,NaN,172.0,0.0,0.0,0.0,0.0
3,Entidad Federativa,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,Objeto del Gasto,0.0,1.0,0.0,0.0,0.0,0.0,NaN
5,Grupo Funcional,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Fuente de Financiamiento,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,Descripción de Concepto,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,Pagado,NaN,NaN,NaN,NaN,0.0,0.0,0.0
9,Descripción de Tipo de Gasto,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [43]:
sums

,Column,2010,2011,2012,2013,2014,2015,2016
0,Ejercido,2.474100e+12,2.695930e+12,2.896331e+12,3.134797e+12,NaN,NaN,NaN
1,Devengado,NaN,NaN,NaN,3.135015e+12,3.426242e+12,3.761997e+12,NaN
2,Aprobado,2.376915e+12,2.538282e+12,2.754868e+12,2.943495e+12,3.334259e+12,3.508463e+12,5.296820e+12
3,Pagado,NaN,NaN,NaN,NaN,3.386609e+12,3.728056e+12,2.707418e+12
4,Modificado,NaN,NaN,NaN,NaN,3.427172e+12,3.763467e+12,2.850453e+12
5,Adefas,NaN,NaN,NaN,NaN,3.694161e+10,3.112265e+10,NaN
6,Ejercicio,NaN,NaN,NaN,NaN,3.424774e+12,3.760422e+12,NaN


In [44]:
merged.sample(n=20) 

,Actividad Institucional,Adefas,Aprobado,Capitulo,Ciclo,Clave de cartera,Concepto,Descripción de Capitulo,Descripción de Concepto,Descripción de Entidad Federativa,...,Modalidad del Programa presupuestario,Modificado,Pagado,Partida Específica,Partida Genérica,Programa Presupuestario,Ramo,Subfunción,Tipo de Gasto,Unidad Responsable
112218,3.6.1.2,NaN,23412.0,2000,2010,NaN,2100,Materiales y suministros,"Materiales de administracion, emision de docum...",NaN,...,M,NaN,NaN,NaN,2106,M.1,21,3.6.1,1,21.W3J
238792,2.6.1.7,NaN,1189.0,1000,2016,0,1400,Servicios personales,Seguridad social,Querétaro,...,J,594.00,0.00,NaN,NaN,J.21,51,2.6.1,1,51.GYN
28440,1.3.4.1,0.0,6359.0,2000,2015,0,2100,Materiales y suministros,"Materiales de administracion, emision de docum...",Distrito Federal,...,O,160.88,160.88,21201,212,O.1,6,1.3.4,1,6.113
45723,2.1.6.2,NaN,4512.0,3000,2016,0,3700,Servicios generales,Servicios de traslado y viáticos,Jalisco,...,M,10760.70,10760.70,NaN,NaN,M.1,16,2.1.6,1,16.134
55558,3.5.6.2,0.0,178329.0,1000,2014,0,1300,Servicios personales,Remuneraciones adicionales y especiales,Durango,...,M,138324.00,138324.00,13201,132,M.1,9,3.5.6,1,9.630
150669,3.2.1.4,NaN,11540.0,3000,2012,0,3900,Servicios generales,Otros servicios generales,Jalisco,...,P,NaN,NaN,39202,392,P.1,15,3.2.1,1,15.134
75033,3.5.1.10,NaN,36000.0,3000,2013,12096480001,3100,Servicios generales,Servicios basicos,Tamaulipas,...,K,NaN,NaN,31401,314,K.31,9,3.5.1,3,9.648
235977,2.3.2.5,NaN,0.0,3000,2016,0,3300,Servicios generales,"Servicios profesionales, cientificos, tecnicos...",Zacatecas,...,M,4205806.00,3796772.00,NaN,NaN,M.2,51,2.3.2,1,51.GYN
147585,3.1.2.5,NaN,3000.0,3000,2012,0,3500,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Zacatecas,...,E,NaN,NaN,35201,352,E.3,14,3.1.2,1,14.152
71871,2.1.4.2,NaN,244562.0,1000,2011,NaN,1700,Servicios personales,Pago de estímulos a servidores públicos,NaN,...,M,NaN,NaN,17102,171,M.1,12,2.1.4,1,12.172


In [45]:
with open(join(ITERATION_FOLDER, BASENAME + '.missing.json')) as file:
    aliases = loads(file.read())
aliases['Descripción de Fuente de Financiamiento']

[{'Actividad Institucional': '3',
  'Aprobado': nan,
  'Ciclo': '2011',
  'Descripción de Fuente de Financiamiento': nan,
  'Descripción de Función': 'Ciencia y Tecnología',
  'Descripción de Grupo Funcional': 'Desarrollo Económico',
  'Descripción de Objeto del Gasto': nan,
  'Descripción de Programa Presupuestario': nan,
  'Descripción de Ramo': 'Consejo Nacional de Ciencia y Tecnología',
  'Descripción de Subfunción': 'Investigación Científica',
  'Descripción de Tipo de Gasto': nan,
  'Descripción de Unidad Responsable': 'Centro de Investigación en Materiales Avanzados, S.C.',
  'Descripción de la Actividad Institucional': nan,
  'Descripción de la modalidad del programa presupuestario': nan,
  'Ejercido': nan,
  'Fuente de Financiamiento': nan,
  'Función': '7',
  'Grupo Funcional': '3',
  'Modalidad del Programa presupuestario': nan,
  'Objeto del Gasto': nan,
  'Programa Presupuestario': nan,
  'Ramo': '38',
  'Subfunción': '1',
  'Tipo de Gasto': nan,
  'Unidad Responsable': '9

In [46]:
breakdown = [
    'Ciclo', 
    'Capitulo', 
    'Concepto', 
    'Partida Genérica',        
    'Partida Específica', 
    'Descripción de Capitulo',
    'Descripción de Concepto', 
    'Descripción de Partida Genérica',
    'Descripción de Partida Específica'
]

merged[breakdown].sample(n=200)

,Ciclo,Capitulo,Concepto,Partida Genérica,Partida Específica,Descripción de Capitulo,Descripción de Concepto,Descripción de Partida Genérica,Descripción de Partida Específica
133742,2012,2000,2400,246,24601,Materiales y suministros,Materiales y articulos de construccion y de re...,Material eléctrico y electrónico,Material eléctrico y electrónico
158910,2014,3000,3500,351,35101,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Conservación y mantenimiento menor de inmuebles,Mantenimiento y conservación de inmuebles para...
226833,2016,1000,1100,NaN,NaN,Servicios personales,Remuneraciones al personal de carácter permanente,NaN,NaN
202306,2015,3000,3100,315,31501,Servicios generales,Servicios basicos,Telefonía celular,Servicio de telefonía celular
200743,2015,3000,3700,372,37204,Servicios generales,Servicios de traslado y viáticos,Pasajes terrestres,Pasajes terrestres nacionales para servidores ...
219076,2012,1000,1300,134,13404,Servicios personales,Remuneraciones adicionales y especiales,Compensaciones,Compensaciones por servicios eventuales
195485,2013,1000,1500,159,15901,Servicios personales,Otras prestaciones sociales y económicas,Otras prestaciones sociales y económicas,Otras prestaciones
54979,2015,2000,2700,272,27201,Materiales y suministros,"Vestuario, blancos, prendas de protección y ar...",Prendas de seguridad y protección personal,Prendas de protección personal
81601,2012,3000,3500,351,35101,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Conservación y mantenimiento menor de inmuebles,Mantenimiento y conservación de inmuebles para...
29184,2011,2000,2700,273,27301,Materiales y suministros,"Vestuario, blancos, prendas de protección y ar...",Artículos deportivos,Artículos deportivos


In [47]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520443 entries, 0 to 239082
Data columns (total 39 columns):
Actividad Institucional                                    1520443 non-null object
Adefas                                                     482560 non-null float64
Aprobado                                                   1520441 non-null float64
Capitulo                                                   1520442 non-null object
Ciclo                                                      1520443 non-null object
Clave de cartera                                           1252602 non-null object
Concepto                                                   1520442 non-null object
Descripción de Capitulo                                    1520442 non-null object
Descripción de Concepto                                    1517818 non-null object
Descripción de Entidad Federativa                          1252430 non-null object
Descripción de Fuente de Financiamiento                  